**ETL Vagas Gupy - Monitor de Oportunidades de Dados**

Este projeto é um pipeline de Engenharia de Dados (ETL) que automatiza a coleta, tratamento e análise de vagas de emprego na plataforma Gupy, focado em tecnologias de dados (Power BI, SQL, Python) em todo o território nacional.

**Funcionalidades**

Extract (Extração): Robô (Web Scraper) desenvolvido com Selenium que simula navegação humana para buscar vagas.
Transform (Transformação):
Classificação automática de localidade (identifica cidades polo como Barueri, Campinas e capitais).
Identificação de ferramentas exigidas (Power BI, SQL, Python, Excel).
Padronização de dados e remoção de duplicatas.
Load (Carga): Conexão via API com o Google Sheets para armazenamento em nuvem.
Dashboard: Os dados alimentam um painel no Power BI para visualização de tendências de mercado.

**Tecnologias Utilizadas**

Linguagem: Python 3.10+
Automação Web: Selenium WebDriver
Manipulação de Dados: Pandas
Cloud/Armazenamento: Google Sheets API (gspread)

**Ambiente de Execução** 
Kaggle Notebooks (Cloud Computing)

**Como Funciona a Lógica de Geolocalização**

O script possui um mapeamento inteligente que corrige inconsistências comuns em descrições de vagas.

Exemplo: Uma vaga listada como "Barueri - SP" ou "Alphaville" é automaticamente categorizada como São Paulo.
Exemplo: Vagas com termos "Home Office" ou "Remote" têm prioridade e são classificadas como Remoto, independente da cidade sede da empresa.

**Como Executar**

Clone o repositório:
git clone [https://github.com/SEU-USUARIO/ETL-VAGAS-GUPY.git](https://github.com/SEU-USUARIO/ETL-VAGAS-GUPY.git)

**Instale as dependências:**
pip install -r requirements.txt

**Configuração de Credenciais:**
É necessário um arquivo service_account.json do Google Cloud Platform para acesso à API do Sheets.

**Nota:**
Por segurança, as credenciais não estão incluídas neste repositório.


In [1]:
!pip install python-dotenv > /dev/null


[notice] A new release of pip is available: 24.3.1 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
print("1. Instalando bibliotecas...")
##!python pip install selenium webdriver-manager gspread oauth2client pandas > /dev/null

import time
import json
import pandas as pd
import gspread
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

def get_driver():
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--window-size=1920,1080')
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36")
    return webdriver.Chrome(options=options)


MAPA_ESTADOS = {
    "Acre": ["acre", " ac ", "-ac", "/ac", "(ac)", "rio branco"],
    "Alagoas": ["alagoas", " al ", "-al", "/al", "(al)", "maceio"],
    "Amapá": ["amapá", "amapa", " ap ", "-ap", "/ap", "(ap)", "macapa"],
    "Amazonas": ["amazonas", " am ", "-am", "/am", "(am)", "manaus"],
    "Bahia": ["bahia", " ba ", "-ba", "/ba", "(ba)", "salvador", "camacari", "feira de santana"],
    "Ceará": ["ceará", "ceara", " ce ", "-ce", "/ce", "(ce)", "fortaleza"],
    "Distrito Federal": ["distrito federal", " df ", "-df", "/df", "(df)", "brasilia", "brasília"],
    "Espírito Santo": ["espírito santo", "espirito santo", " es ", "-es", "/es", "(es)", "vitoria", "vitória", "vila velha", "serra - es"],
    "Goiás": ["goiás", "goias", " go ", "-go", "/go", "(go)", "goiania", "aparecida de goiania"],
    "Maranhão": ["maranhão", "maranhao", " ma ", "-ma", "/ma", "(ma)", "sao luis"],
    "Mato Grosso": ["mato grosso", " mt ", "-mt", "/mt", "(mt)", "cuiaba"],
    "Mato Grosso do Sul": ["mato grosso do sul", " ms ", "-ms", "/ms", "(ms)", "campo grande"],
    "Minas Gerais": ["minas gerais", " mg ", "-mg", "/mg", "(mg)", "belo horizonte", "bh", "uberlandia", "contagem", "betim", "juiz de fora"],
    "Pará": ["pará", "para ", " pa ", "-pa", "/pa", "(pa)", "belem"],
    "Paraíba": ["paraíba", "paraiba", " pb ", "-pb", "/pb", "(pb)", "joao pessoa"],
    "Paraná": ["paraná", "parana", " pr ", "-pr", "/pr", "(pr)", "curitiba", "londrina", "maringa", "sao jose dos pinhais"],
    "Pernambuco": ["pernambuco", " pe ", "-pe", "/pe", "(pe)", "recife", "jaboatao", "olinda"],
    "Piauí": ["piauí", "piaui", " pi ", "-pi", "/pi", "(pi)", "teresina"],
    "Rio de Janeiro": ["rio de janeiro", " rj ", "-rj", "/rj", "(rj)", "niteroi", "niterói", "duque de caxias", "nova iguacu"],
    "Rio Grande do Norte": ["rio grande do norte", " rn ", "-rn", "/rn", "(rn)", "natal"],
    "Rio Grande do Sul": ["rio grande do sul", " rs ", "-rs", "/rs", "(rs)", "porto alegre", "caxias do sul", "canoas"],
    "Rondônia": ["rondônia", "rondonia", " ro ", "-ro", "/ro", "(ro)", "porto velho"],
    "Roraima": ["roraima", " rr ", "-rr", "/rr", "(rr)", "boa vista"],
    "Santa Catarina": ["santa catarina", " sc ", "-sc", "/sc", "(sc)", "florianopolis", "blumenau", "joinville", "sao jose - sc"],
    "São Paulo": ["são paulo", "sao paulo", " sp ", " sp,", "-sp", " - sp", "/sp", "/ sp", "(sp)", "barueri", "alphaville", "osasco", "campinas", "guarulhos", "sbc", "bernardo", "santo andre", "sao caetano", "jundiai", "sorocaba", "ribeirao preto", "sao jose dos campos"],
    "Sergipe": ["sergipe", " se ", "-se", "/se", "(se)", "aracaju"],
    "Tocantins": ["tocantins", " to ", "-to", "/to", "(to)", "palmas"]
}

def classificar_local_brasil(texto_completo):
    texto_lower = texto_completo.lower()
    
    if "remoto" in texto_lower or "remote" in texto_lower or "home office" in texto_lower:
        return "Remoto"
    
    for estado_nome, termos in MAPA_ESTADOS.items():
        for termo in termos:
            if termo in texto_lower:
                return estado_nome
                
    return "Outros / Não Identificado"

def identificar_ferramentas(texto_completo):
    texto_lower = texto_completo.lower()
    ferramentas = []
    
    if "power bi" in texto_lower or "pbi" in texto_lower or "powerbi" in texto_lower:
        ferramentas.append("Power BI")
    if "sql" in texto_lower:
        ferramentas.append("SQL")
    if "python" in texto_lower:
        ferramentas.append("Python")
    if "excel" in texto_lower:
        ferramentas.append("Excel")
    if "etl" in texto_lower:
        ferramentas.append("ETL")        
    if "dados" in texto_lower:
        ferramentas.append("Dados")        
        
    return ", ".join(ferramentas) if ferramentas else "Geral/Outras"

termos_busca = ["Power BI", "SQL", "Python", "ETL", "Dados"]

def buscar_vagas_brasil_corrigido():
    driver = get_driver()
    lista_final = []
    
    print(f"--- INICIANDO ROBÔ (GEOGRAFIA CORRIGIDA) ---")
    
    for termo in termos_busca:
        print(f"\n>> 🔍 Pesquisando: '{termo}'...")
        
        try:
            driver.get("https://portal.gupy.io/")
            time.sleep(3)
            try: driver.find_element(By.XPATH, "//button[contains(text(), 'Aceitar')]").click()
            except: pass

            input_busca = driver.find_element(By.TAG_NAME, "input")
            input_busca.send_keys(Keys.CONTROL + "a")
            input_busca.send_keys(Keys.DELETE)
            time.sleep(0.5)
            input_busca.send_keys(termo)
            time.sleep(1)
            input_busca.send_keys(Keys.RETURN)
            
            print("   ⏳ Carregando...")
            time.sleep(8)
            driver.execute_script("window.scrollTo(0, 1000);")
            
            cards = driver.find_elements(By.CSS_SELECTOR, 'li[data-testid="job-list-item"]')
            if len(cards) == 0:
                cards = driver.find_elements(By.XPATH, "//a[contains(@href, '/job/')]")

            print(f"   📋 Cards analisados: {len(cards)}")
            
            for card in cards:
                try:
                    texto_completo = card.text
                    
                    linhas = texto_completo.split('\n')
                    empresa = linhas[0] if len(linhas) > 0 else "Confidencial"
                    titulo = linhas[1] if len(linhas) > 1 else "Vaga"
                    
                    if card.tag_name == 'a': link = card.get_attribute('href')
                    else: link = card.find_element(By.TAG_NAME, 'a').get_attribute('href')

                    local_real = classificar_local_brasil(texto_completo)
                    tools = identificar_ferramentas(texto_completo)
                    
                    lista_final.append({
                        "Vaga": titulo,
                        "Empresa": empresa,
                        "Local": local_real,
                        "Ferramentas": tools,
                        "Link": link,
                        "Data_Coleta": pd.Timestamp.now().strftime('%Y-%m-%d')
                    })
                except:
                    continue
                    
        except Exception as e:
            print(f"   Erro: {e}")
            driver.quit()
            driver = get_driver()
            
    driver.quit()
    return pd.DataFrame(lista_final)

df = buscar_vagas_brasil_corrigido()

if not df.empty:
    df_final = df.drop_duplicates(subset=['Link'])
    
    print("\n--- AMOSTRA DA CLASSIFICAÇÃO ---")
    print(df_final[['Vaga', 'Local']].head(10)) 
    print(f"\nTotal: {len(df_final)}")

    try:
        gcp_account_service = os.getenv("GCP_ACCOUNT_SERVICE")
        if gcp_account_service:
                gcp_account_service = json.loads(gcp_account_service)

        gc = gspread.service_account_from_dict(gcp_account_service)
        sh = gc.open("Vagas_Gupy_RJ") 
        ws = sh.sheet1
        ws.append_rows(df_final.values.tolist())
        print("🚀 SUCESSO! Dados enviados.")
    except Exception as e:
        print(f"Erro Sheets: {e}")
else:
    print("Nenhuma vaga encontrada.")


1. Instalando bibliotecas...
--- INICIANDO ROBÔ (GEOGRAFIA CORRIGIDA) ---

>> 🔍 Pesquisando: 'Power BI'...
   ⏳ Carregando...
   📋 Cards analisados: 10

>> 🔍 Pesquisando: 'SQL'...
   ⏳ Carregando...
   📋 Cards analisados: 10

>> 🔍 Pesquisando: 'Python'...
   ⏳ Carregando...
   📋 Cards analisados: 10

>> 🔍 Pesquisando: 'ETL'...
   ⏳ Carregando...
   📋 Cards analisados: 3

>> 🔍 Pesquisando: 'Dados'...
   ⏳ Carregando...
   📋 Cards analisados: 10

--- AMOSTRA DA CLASSIFICAÇÃO ---
                                                Vaga  \
0                            ANALISTA DE POWER BI PL   
1                        10795750 - POWER BI ANALYST   
2                        Analista de Power BI Júnior   
3                               Analista Power BI Sr   
4    Engenheiro de Dados (Power BI) I Modelo Hibrido   
5            Desenvolvedor Power Automate | Power BI   
6  Data Visualization | Setor Bancário | Consigna...   
7  Analista Júnior de Power BI - AMD GSC (Automot...   
8  TECNICO QUA

In [9]:
# =========================================
# LOAD NO MONGODB ATLAS (FREE)
# =========================================

!pip install pymongo > /dev/null

from pymongo import MongoClient

# STRING DE CONEXÃO (FORNECIDA POR VOCÊ)
MONGO_URI = os.getenv("MONGO_URI")

# CONEXÃO
client = MongoClient(MONGO_URI)

# BANCO E COLEÇÃO
db = client["etl_gupy"]
collection = db["vagas"]

# CONVERTE DATAFRAME PARA LISTA DE DOCUMENTOS
registros = df_final.to_dict(orient="records")

# UPSERT PARA EVITAR DUPLICIDADE (BASEADO NO LINK)
for vaga in registros:
    collection.update_one(
        {"Link": vaga["Link"]},   # chave única
        {"$set": vaga},
        upsert=True
    )

print(f"✅ {len(registros)} vagas inseridas/atualizadas no MongoDB Atlas")



[notice] A new release of pip is available: 24.3.1 -> 26.0
[notice] To update, run: pip install --upgrade pip
✅ 38 vagas inseridas/atualizadas no MongoDB Atlas


In [10]:
# ================================
# MAPA DE VAGAS - SEM GEOCODING ONLINE
# MongoDB Atlas + Kaggle
# ================================

!pip install --no-deps pymongo folium pandas

from pymongo import MongoClient
import pandas as pd
import folium

# ----------------
# Conexão MongoDB
# ----------------
MONGO_URI = os.getenv("MONGO_URI")

client = MongoClient(MONGO_URI)

db = client["etl_gupy"]       
collection = db["vagas"]    

# ----------------
# Carregar dados
# ----------------
docs = list(collection.find({}, {"_id": 0}))
df = pd.DataFrame(docs)

# ----------------
# Limpeza
# ----------------
df = df.dropna(subset=["Local", "Empresa", "Vaga"])
df = df[~df["Local"].str.contains("Remoto", case=False, na=False)]

# ----------------
# Coordenadas fixas (Brasil)
# ----------------
COORDENADAS_BR = {
    "São Paulo": (-23.5505, -46.6333),
    "Rio de Janeiro": (-22.9068, -43.1729),
    "Belo Horizonte": (-19.9167, -43.9345),
    "Curitiba": (-25.4296, -49.2713),
    "Porto Alegre": (-30.0346, -51.2177),
    "Florianópolis": (-27.5949, -48.5482),
    "Campinas": (-22.9099, -47.0626),
    "Recife": (-8.0476, -34.8770),
    "Salvador": (-12.9714, -38.5014),
    "Fortaleza": (-3.7172, -38.5433),
    "Brasília": (-15.7939, -47.8828)
}

# Normalizar local
df["Local"] = df["Local"].str.strip()

# Aplicar coordenadas
df["lat"] = df["Local"].map(lambda x: COORDENADAS_BR.get(x, (None, None))[0])
df["lon"] = df["Local"].map(lambda x: COORDENADAS_BR.get(x, (None, None))[1])

df = df.dropna(subset=["lat", "lon"])

# ----------------
# Criar mapa
# ----------------
mapa = folium.Map(
    location=[-14.2350, -51.9253],
    zoom_start=4,
    tiles="OpenStreetMap"
)

for _, row in df.iterrows():
    popup = f"""
    <b>Empresa:</b> {row['Empresa']}<br>
    <b>Vaga:</b> {row['Vaga']}<br>
    <b>Ferramentas:</b> {row.get('Ferramentas', 'N/A')}<br>
    <b>Local:</b> {row['Local']}
    """
    
    folium.Marker(
        location=[row["lat"], row["lon"]],
        popup=popup,
        icon=folium.Icon(icon="briefcase", prefix="fa")
    ).add_to(mapa)

mapa



[notice] A new release of pip is available: 24.3.1 -> 26.0
[notice] To update, run: pip install --upgrade pip
